## Voice Activity Detector (VAD) in Python

In [1]:
import torch
import pyaudio
import numpy as np
from IPython.display import Audio
from pprint import pprint

In [79]:
# Load the JIT file
jit_model = torch.jit.load("./vad_model/vad.jit")

In [ ]:
# use pyaudio to get audio stream
p = pyaudio.PyAudio()

stream = p.open(format=pyaudio.paInt16,
                channels=1,
                rate=16000,
                input=True,
                frames_per_buffer=8000)

while True:
    data = stream.read(8000)

    # Convert audio data from bytes to numpy array
    data = np.frombuffer(data, dtype=np.float32)

    # Convert numpy array to a tensor
    data_tensor = torch.from_numpy(data)
    data_tensor = torch.reshape(data_tensor, (1, 4000))
    data_tensor = torch.nan_to_num(data_tensor)
    print(data_tensor)
    
    # Send data to AI model
    output = jit_model(data_tensor)

    print(output)

    break

stream.stop_stream()
stream.close()
p.terminate()

In [2]:
# use pyaudio to get audio stream
torch.set_num_threads(1)

speech_probs = []
window_size_samples = 512 # use 256 for 8000 Hz model
SAMPLING_RATE = 16000

model, utils = torch.hub.load(repo_or_dir='snakers4/silero-vad',
                              model='silero_vad',
                              force_reload=True)

(get_speech_timestamps,
 _, read_audio,
 *_) = utils

p = pyaudio.PyAudio()

stream = p.open(format=pyaudio.paInt16,
                channels=1,
                rate=16000,
                input=True,
                frames_per_buffer=SAMPLING_RATE)


while True:
    data = stream.read(SAMPLING_RATE)

    chunk = data
    speech_prob = model(chunk, SAMPLING_RATE).item()
    speech_probs.append(speech_prob)

    break

stream.stop_stream()
stream.close()
p.terminate()

Downloading: "https://github.com/snakers4/silero-vad/archive/master.zip" to C:\Users\Hisku/.cache\torch\hub\master.zip


FileNotFoundError: Could not find module 'C:\Users\Hisku\AppData\Local\Programs\Python\Python39\Lib\site-packages\torchaudio\lib\libtorchaudio.pyd' (or one of its dependencies). Try using the full path with constructor syntax.